In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
import random as rn
import gc
import logging
tqdm.pandas()
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1017)
rn.seed(1017)
path="data/"
os.listdir("data/")

In [ ]:
# 读入数据（需加速）
def get_age_data():
    train_data = pd.read_csv(path + 'age_train.csv', header=None)
    test_data = pd.read_csv(path + 'age_test.csv', header=None)
    data = pd.concat([train_data, test_data], axis=0, sort=False).fillna(-1)
    data.columns = ['uId', 'age_group']
    return data

def get_user_app_actived():
    data = pd.read_csv(path + 'user_app_actived.csv', header=None)
    data.columns = ['uId', 'appId']
    return data

def get_user_behavior_info():
    data = pd.read_csv(path + 'user_behavior_info.csv', header=None)
    data.columns = ['uId', 'bootTimes', 'AFuncTimes', 'BFuncTimes', 'CFuncTimes',
                   'DFuncTimes', 'EFuncTimes', 'FFuncTimes', 'FFuncSum']
    return data

def get_user_basic_info():
    data = pd.read_csv(path + 'user_basic_info.csv', header=None)
    data.columns = ['uId', 'gender', 'city', 'prodName', 'ramCapacity', 
                   'ramLeftRation', 'romCapacity', 'romLeftRation', 'color',
                   'fontSize', 'ct', 'carrier', 'os']
    return data

def get_app_info():
    data = pd.read_csv(path + 'app_info.csv', header=None)
    data.columns = ['appId', 'category']
    return data

# 测试的时候用True
# 提特征改用False
def get_user_app_usage(less_data=False):
    if less_data:
        reader = pd.read_csv(path + 'user_app_usage.csv', chunksize=2000000)
        for i in reader:
            data = i
            break
    else:
        data = pd.read_csv(path + 'user_app_usage.csv', header=None)
    data.columns = ['uId', 'appId', 'duration', 'times', 'use_date']
    return data

In [6]:
packtime_all = get_user_app_usage(less_data=False)
packtime_all.columns = ['uId', 'app', 'peroid', 'times', 'start']
del packtime_all['peroid']
packtime_all = packtime_all.sort_values('start')
packtime_all = packtime_all.sort_values('times', ascending=False)

train_data = pd.read_csv(path + 'age_train.csv', header=None)
test_data = pd.read_csv(path + 'age_test.csv', header=None)
del train_data[1]
train_data.columns = ['uId']
test_data.columns = ['uId']

d1 = train_data[:500000]
d2 = train_data[500000:1000000]
d3 = train_data[1000000:1500000]
d4 = train_data[1500000:]
d5 = test_data

def set_first_times(row):
    return ' '.join(list(row['app']))

df_value = []
for i in tqdm([d1, d2, d3, d4, d5]):
    i = pd.merge(i, packtime_all, on='uId', how='left')
    i =i.fillna(0)
    i['app'] = i['app'].astype(str)
    group_data = i.groupby(['uId', 'start']).progress_apply(lambda row: set_first_times(row)).reset_index()
    group_data.columns = ['uId', 'start', 'app']
    group_data =  group_data.groupby('uId').progress_apply(lambda row:set_first_times(row)).reset_index()
    df_value.append(group_data)
data = pd.concat([df_value[0], df_value[1], df_value[2], df_value[3], df_value[4]], axis=0, sort=False)
data.to_csv('data/set_series_1.csv', index=False, header=None)

NameError: name 'get_user_app_usage' is not defined